In [14]:
from testing_criterions import CPTDigraphManager
from diblob import DigraphManager
import re


def extract_k_test_cases(test_cases, delimiter = ',', source='S', sink='T', k=1):

    test_cases_string = ','.join([elem[0] for elem in test_cases]) + f",{source}"
    delimiters = "".join([f"{sink},{sink}{source}{number},{source}|" for number in range(k)])

    test_cases = re.split(delimiters[:-1], test_cases_string)
    test_cases = [test_case for test_case in test_cases if test_case]
    test_cases = [f"{source},{test_case.replace(source, '').replace(sink, '')[1:-1]},{sink}".replace(',', delimiter) 
                  for test_case in test_cases]
    return test_cases

In [33]:
def generate_test_cases(digraph_manager, 
                        source='S', 
                        sink='T', 
                        cost_function={}, 
                        additional_edge_cost=2, 
                        default_cost=1,
                        k=1, 
                        delimiter=','):
    
    if source in digraph_manager or sink in digraph_manager:
        raise Exception('sink/source should be unique over digraph namespace!')
    
    dict_json_representation = dict(digraph_manager(digraph_manager.root_diblob_id))
    cpt = CPTDigraphManager(dict_json_representation, cost_function={})

    minimal_elements, maximal_elements = cpt.get_edge_elements()
    
    nodes_to_add = [sink + source + str(number) for number in range(k)] + [sink, source]
    cpt.add_nodes(*nodes_to_add)

    minimal_edges = [(source, node_id) for node_id in minimal_elements]
    maximal_edges = [(node_id, sink) for node_id in maximal_elements]
    
    if not minimal_edges or not maximal_edges:
        raise Exception('Should be at least minimal/maximal element in the diblob!')
    
    sink_source_edges = [(sink, sink + source + str(number)) for number in range(k)] +\
                        [(sink + source + str(number), source) for number in range(k)]
    
    new_edges = minimal_edges + maximal_edges + sink_source_edges
    cpt.connect_nodes(*new_edges)

    cost_function_update_dict = {edge_id: cost_function.get(edge_id, 1)*default_cost
                                 for edge_id in cpt.edges}

    for number in range(k):
        cost_function_update_dict[(sink, sink + source + str(number))] = additional_edge_cost - 1
        cost_function_update_dict[(sink + source + str(number), source)] = 1

    cpt.update_cost_function(cost_function_update_dict)

    cpt, cost = cpt.compute_cpt(start_node=source)
    return extract_k_test_cases(cpt, delimiter, source, sink, k), cost


In [34]:
cpt_digraph_manager = DigraphManager({"B0": {}})
cpt_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I')
cpt_digraph_manager.connect_nodes(('A', 'C'), ('B', 'C'), ('C', 'D'), ('C', 'E'), ('C', 'F'),
                       ('D', 'I'), ('D', 'G'), ('E', 'G'), ('E', 'H'), ('F', 'H'))
                       

# Case 5
number_of_nodes = len(cpt_digraph_manager.nodes)
weight = number_of_nodes * (number_of_nodes + 1) / 2

test_cases, cost = generate_test_cases(digraph_manager=cpt_digraph_manager,
                                       default_cost=weight)
print("Case 5: Minimal total cost (any number of test cases)")
print(f"{test_cases=}")
print(f"{cost=}\n")

Case 5: Minimal total cost (any number of test cases)
test_cases=['S,B,C,E,G,T', 'S,A,C,D,G,T', 'S,A,C,D,I,T', 'S,A,C,F,H,T', 'S,A,C,E,H,T']
cost=1135.0



In [36]:
cpt2_digraph_manager = DigraphManager({"B0": {}})
cpt2_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G')
cpt2_digraph_manager.connect_nodes(('A', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'F'),
                       ('D', 'F'), ('E', 'F'), ('F', 'B'), ('F', 'G'))

# Case 1
weight = number_of_nodes * (number_of_nodes + 1) / 2

test_cases, cost = generate_test_cases(digraph_manager=cpt2_digraph_manager,
                                       default_cost=weight,
                                       additional_edge_cost=weight**3)
print("Case 1: Minimal total number of test cases (without cost)")
print(f"{test_cases=}")
print(f"{cost=}\n")

test_cases, cost = generate_test_cases(digraph_manager=cpt2_digraph_manager,
                                       default_cost=weight,
                                       cost_function={('F', 'B'): 20})
print("Case 5: Minimal total cost (any number of test cases)")
print(f"{test_cases=}")
print(f"{cost=}\n")

Case 1: Minimal total number of test cases (without cost)
test_cases=['S,A,B,C,F,B,E,F,B,D,F,G,T']
cost=91665.0

Case 5: Minimal total cost (any number of test cases)
test_cases=['S,A,B,C,F,G,T', 'S,A,B,E,F,B,D,F,G,T']
cost=1534.0



In [39]:

cpt3_digraph_manager = DigraphManager({"B0": {}})
cpt3_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G')
cpt3_digraph_manager.connect_nodes(('A', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'F'),
                       ('D', 'F'), ('E', 'F'), ('F', 'G'))

# Case 3
K=5
test_cases, cost = generate_test_cases(digraph_manager=cpt2_digraph_manager,
                                       default_cost=weight,
                                       additional_edge_cost=weight**2*K,
                                       k=K)

print("Case 4: Set number of test cases")
print(f"{test_cases=}")
print(f"{cost=}\n")

Case 4: Set number of test cases
test_cases=['S,A,B,C,F,G,T', 'S,A,B,D,F,G,T', 'S,A,B,D,F,G,T', 'S,A,B,D,F,G,T', 'S,A,B,E,F,B,D,F,G,T']
cost=52110.0

